<div style="border-bottom:1px dotted black;">
<font size="4">
<font color = D4C36A> **OpenStreetMap Sample Project  
Data Wrangling with MongoDB**</font>    
_Jason Medina DAND_
</div>

[Link to OpenStreetMap Area for Culver City](https://www.openstreetmap.org/relation/3492137)  
[Link to mapzen extracts page](https://mapzen.com/data/metro-extracts/)

<div style="border-bottom:1px dotted black;">
<font size="4">
<font color = 'black'>

<UL>
1) Problems Encountered in the Map  
<L1><UL> Audit Street Names  
<L1> Inconsistent Postal Codes</UL>  2) Data Overview  
3) Additional Ideas  
 <UL><L1> Contributor statistics and suggestion  
 <L1> Additional data exploration with MongoDB  
 <L1> Conclusion</UL>
</UL>
 </font>
 </div>
 
 For this project, the websites below were reviewed to assist with the report: <br>
 [Sample MongoDB project](https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md)<br>
 [Fun read, awesome example](https://jameskao.me/everything-you-need-to-know-about-facebooks-flux-graphql-and-relay/)<br>
 [Excellent examples and code](https://github.com/allanbreyes/udacity-data-science/tree/master/p2)<br>
 [Good examples and code](https://github.com/paul-reiners/udacity-data-wrangling-mongo-db)<br>
 
<div style="border-bottom:1px dotted black;">
<font size="4">
<font color = 'black'>
 

### 1. Problems encountered in the map 

#### To improve data quality, we start with a simple data audit like we saw in the OSM case study by matching the last token in the string optionally ending in a period.  The audit.py has a list of unexpected street types and postcodes.  The audit is necessary to prepare for executing data.py to output the json file for MongoDB   
+ over abbreviated street names (e.g. "Blvd." and "Bvd" or "ave" and "avenue")
+ postal codes starting with CA or not five digits to exclude
<div style="border-bottom: 1px dotted black;">
</div>

#### Street name abbreviations
The audit.py file looks for street names not ending in an expected value, then prints a list of street names with updates.

   >_South Fairfax Ave => South Fairfax Avenue_

   >_north la brea avenue => North La Brea Avenue_

   >_W Pico Blvd => W Pico Boulevard_ 

Applying the title method capitalizes the first word in each street name.  Replacing the cardinal directions (i.e. N,S,E,W) with the full name would improve data quality.

#### Postal Code formats
The zip_code_audit.py file looks for postcode values that are not five digits nor start with '90'.  The zipcodes look clean, however we could improve by data consistency by excluding these records or removing the CA programmatically.

> <i> CA 90045 1 <br>
> CA 90291 1  
> CA 90405 1  
> CA 90404 1  
> CA 90034 1   
> CA 90036 1  
> CA 90024 1   
> 91108 1   
> CA 90232 1</i>
<div style="border-bottom: 1px dotted black;">
</div>

### 2. Data Overview 

Summary statistics about data - see data.py file for complete code:

>1) Original xml file size: 603MB    
    <i>print(os.path.getsize(os.path.join(path, "culver-city_ca.osm.json"))/1024/1024)  

>2) New json file size: 903MB  
    <i>print(os.path.getsize(filename)/1024/1024)

>3) Document count: 3027238  
    <i>print(collection.count())

>4) Nodes count: 2757124  
    <i>print(collection.find({"type":"node"}).count())

>5) Ways count: 270091  
    <i>print(collection.find({"type":"way"}).count())

>6) UU count: 569  
    <i>print(len(collection.group(["created.uid"], {}, {"count":0}, "function(o, p){p.count++}")))
<div style="border-bottom: 1px dotted black;">
</div>

### 3. Additional Ideas

With the data in MongoDB, let's look at some other questions and discuss additional ideas.
 
#### 1) User contributions as ratio, top ten Users 
>```
pipeline = [{"$group":{"_id": "$created.user",
                       "count": {"$sum": 1}}},
            {"$project": {"ratio": {"$divide" :["$count"\
            ,collection.find().count()]}}},
            {"$sort": {"ratio": -1}},
            {"$limit": 10}]
result = collection.aggregate(pipeline)
for doc in result:
    pprint.pprint(doc)
```
<b>User contributions as ratio of total contributions    
{u'_id': u'schleuss_imports', u'ratio': 0.1722556997500692}  
{u'_id': u'manings_labuildings', u'ratio': 0.12147277485285267}  
{u'_id': u'calfarome_labuilding', u'ratio': 0.11513069008779621}  
{u'_id': u'ridixcr_import', u'ratio': 0.10187273019167968}  
{u'_id': u'kari, totp_labuildings', u'ratio': 0.05909347068185587}  
{u'_id': u'kingrollo', u'ratio': 0.05351049372398206}  
{u'_id': u'dannykath_labuildings', u'ratio': 0.04586623185887598}  
{u'_id': u'Luis36995_labuildings', u'ratio': 0.039377478744651064}  
{u'_id': u'RichRico_labuildings', u'ratio': 0.03589146277894239}  
{u'_id': u'schleuss', u'ratio': 0.03488097070663093}</b>      

> Note first and last indices contain Schleuss.  A quick internet search leads me to suspect this could be data from Jon Schleuss, data viz artist for the LA times.  I could not come up with any ideas for kingrollo or the users ending in _labuildings.

#### 2) Top 20 sources
>```# top 20 sources:
pipeline = [{"$match":{"source":{"$exists":1}}},
            {"$group":{"_id": "$source",
                       "count": {"$sum": 1}}},
            {"$sort": {"count": -1}},
            {"$limit":20}]
result = collection.aggregate(pipeline)
for doc in result:
    pprint.pprint(doc)  
```  
<br><b> Top 20 sources: <br>
{u'_id': u'usgs_imagery', u'count': 541}  
{u'_id': u'TIGER, Bing', u'count': 432}  
{u'_id': u'Bing', u'count': 427}  
{u'_id': u'Yahoo!, local knowledge', u'count': 278}  
{u'_id': u'survey;image;usgs_imagery', u'count': 223}  
{u'_id': u'Yahoo', u'count': 195}  
{u'_id': u'Yahoo,TIGER', u'count': 161}  
{u'_id': u'bing_imagery_0.06m_200801', u'count': 124}  
{u'_id': u'survey, image, usgs_imagery', u'count': 123}  
{u'_id': u'USGS Geonames', u'count': 107}  
{u'_id': u'survey', u'count': 95}  
{u'_id': u'Bing, local knowledge', u'count': 85}  
{u'_id': u'yahoo_imagery', u'count': 72}  
{u'_id': u'Yahoo!, Bing, local knowledge', u'count': 62}  
{u'_id': u'survey;image;usgs_imagery;CDOT', u'count': 39}  
{u'_id': u'bing_imagery_0.06m_200801;LACA', u'count': 31}  
{u'_id': u'Bing, TIGER', u'count': 26}  
{u'_id': u'usgs_imagery;survey;image', u'count': 17}  
{u'_id': u'bing', u'count': 13}  
{u'_id': u'Los Angeles Fire Department', u'count': 12}</b>

>  Where a source exists, the usgs_imagery appears twice in the top 5, and multiple times overall.  The other sources include Yahoo, Bing and even the LA Fire Department. TIGER stands for Topologically Integrated Geographic Encoding and Referencing, so it makes sense that we see this as a source.

#### 3) Amenities near me
>```
pipeline = [{"$match":{"amenity":{"$exists":1}, "name":{"$exists":1}}},  
            {"$group":{"_id":"$amenity", "count":{"$sum":1}}},
            {"$sort":{"count":-1}},
            {"$limit":30}]
result = collection.aggregate(pipeline)
for doc in result:
    pprint.pprint(doc)
```<br>

><b>Amenities near me:<br>
{u'_id': u'restaurant', u'count': 206}  
{u'_id': u'school', u'count': 192}  
{u'_id': u'place_of_worship', u'count': 181}  
{u'_id': u'fast_food', u'count': 120}  
{u'_id': u'cafe', u'count': 93}  
{u'_id': u'hospital', u'count': 54}  
{u'_id': u'parking', u'count': 51}  
{u'_id': u'fuel', u'count': 42}  
{u'_id': u'library', u'count': 36}  
{u'_id': u'bank', u'count': 31}  
{u'_id': u'post_office', u'count': 28}  
{u'_id': u'pharmacy', u'count': 24}  
{u'_id': u'fire_station', u'count': 23}  
{u'_id': u'theatre', u'count': 20}  
{u'_id': u'bar', u'count': 16}  
{u'_id': u'cinema', u'count': 12}  
{u'_id': u'police', u'count': 10}
.  
.  
.  
{u'_id': u'ice_cream', u'count': 4}  
{u'_id': u'clinic', u'count': 4}  
{u'_id': u'public_building', u'count': 3}  </b>

>  There are 206 restaurants found, note fast food and cafe are also in the top 5.  The restaurants can be further grouped by cuisine type.

#### 4) Restaurant amenity grouped by cuisine type
```pipeline = [{"$match":{"amenity":{"$exists":1} \
            , "cuisine":{"$exists":1}}}, 
            {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":10}]
result = collection.aggregate(pipeline)
for doc in result:
    pprint.pprint(doc)
```  
<b><br>Restaurant amenties by cusine type:
{u'_id': u'mexican', u'count': 28}  
{u'_id': u'burger', u'count': 23}  
{u'_id': u'coffee_shop', u'count': 22}  
{u'_id': u'pizza', u'count': 14}  
{u'_id': u'italian', u'count': 13}  
{u'_id': u'sandwich', u'count': 12}  
{u'_id': u'american', u'count': 12}  
{u'_id': u'chinese', u'count': 8}  
{u'_id': u'japanese', u'count': 7}  
{u'_id': u'sushi', u'count': 7}  </b>

> Where the cuisine type exists, for any amenity, we see the most common cuisine is Mexican.  Note Sushi might be too specific, as Sushi is a type of Japanese food; same is true for pizza however I can understand the desire for detail.  The chinese number looks low to me, so there could be an opportunity to add data.
<br><br>

<div style="border-bottom: 1px dotted black;">
</div>

#### <font color = grey> Conclusion: <br><br> Other than over abbreviated street names the data looks complete.  This is not unexpected since Culver City is in the major metro area Los Angeles, California.  Interesting to see a columnist from the LA times contributing 20% of the data via two user names. 

Additional excercises could include reformatting zip codes and cardinal directions though I belive this data is clean enough for this excercise.  Adding some sort of data validation or gamification for human interaction could prove fun.  For example, find amenities such as restaurant, cafe or fast food without cuisine types then ping nearby users to contribute information for points or some sort of recognition award.  There is no uniform format for operating hours, finding a way to validate this data would allow users to identify if places were open or not.  However coming up with methods to standardize formats or obscure personal identifying information may be a challenge.  Nor is it easy to see where or how often updates occur.  

Overall this OSM data set is clean and very rich.  I hope to make this an onging project where, with the goal of submitting data to the OSM project.  Stay tuned! </font>   